In [1]:
from google.colab import drive
drive.mount('/content/drive')
# Nikhil's working path
%cd drive/My\ Drive/'Captstone(297)'/data

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1auWpfpINZvmJbl0I50kYmpS29bbnesqp/Captstone(297)/data


In [ ]:
# # Selina's working path
# %cd drive/My\ Drive/2020_Fall/'Captstone(297)'/data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', None)

In [ ]:
df = pd.read_pickle("all_webdata_filtered.pkl")
listing_data = pd.read_pickle('LOG_siamese_data_11_12.pkl')

In [ ]:
df.rename({'rexurl':'rexUrl'},axis=1,inplace=True)

In [ ]:
df_web_la = df.copy()

print('total LA web interactions',df_web_la.shape)

total LA web interactions (4617883, 6)


In [ ]:
# Convert eventstamp to datetime
df_web_la['event_stamp'] = pd.to_datetime(df_web_la['event_stamp'])

#sort by ip and eventstamp
df_web_la.sort_values(['ip','event_stamp'],ascending=True,inplace=True)

In [ ]:
ip_url = df_web_la.groupby(['ip','rexUrl'])['event_stamp'].count().reset_index()

In [ ]:
ip_url['event_stamp'] = ip_url['event_stamp'].apply(lambda x: 1 if x>0 else 0)
ip_url.rename({'event_stamp':'score'},axis=1,inplace=True)

In [ ]:
ip_url.sort_values('ip').head()

,ip,rexUrl,score
0,1.0.84.114,28710-indies-ln-canyon-country,1
1,1.125.109.121,23552-park-south-st,1
2,1.125.110.139,9235-garibaldi-ave,1
3,1.128.106.121,1050-trevecca-pl,1
4,1.128.106.121,9323-alcott-st-apt-101,1


# temporal traintest split

In [ ]:
ip_listing = df_web_la[['ip','rexUrl']].drop_duplicates().reset_index(drop=True)

In [ ]:
# users whov seen more than 1 listing
print(sum(ip_listing.groupby('ip')['rexUrl'].count().values>1),'more than 1')
print(sum(ip_listing.groupby('ip')['rexUrl'].count().values>2),'more than 2')
print(sum(ip_listing.groupby('ip')['rexUrl'].count().values>3),'more than 3')
print(sum(ip_listing.groupby('ip')['rexUrl'].count().values>4),'more than 4')

27933 more than 1
14244 more than 2
8663 more than 3
5809 more than 4


In [ ]:
# gsr = ip_listing.groupby('ip')['rexUrl'].count()
# gsr[gsr>5]

In [ ]:
for i in range(10):
  print(f'see {i} listing, split', list(range(0,int(np.ceil(i*0.7)))) ,  list(range(int(np.ceil(i*0.7)),i)))

see 0 listing, split [] []
see 1 listing, split [0] []
see 2 listing, split [0, 1] []
see 3 listing, split [0, 1, 2] []
see 4 listing, split [0, 1, 2] [3]
see 5 listing, split [0, 1, 2, 3] [4]
see 6 listing, split [0, 1, 2, 3, 4] [5]
see 7 listing, split [0, 1, 2, 3, 4] [5, 6]
see 8 listing, split [0, 1, 2, 3, 4, 5] [6, 7]
see 9 listing, split [0, 1, 2, 3, 4, 5, 6] [7, 8]


In [ ]:
train_g = ip_listing.groupby('ip')['rexUrl'].apply(lambda x: list(x[0:int(np.ceil(len(x)*0.7))]))

In [ ]:
test_g = ip_listing.groupby('ip')['rexUrl'].apply(lambda x: list(x[int(np.ceil(len(x)*0.7)):]))
# test_g['104.32.235.163']

In [ ]:
test_la = pd.DataFrame(test_g).reset_index().explode('rexUrl')
test_la.dropna(inplace=True)

train_la = pd.DataFrame(train_g).reset_index().explode('rexUrl')
#SUBSET TRAIN TO ONLY INCLUDE USERS IN TEST
train_la = train_la[train_la.ip.isin(set(test_la.ip))].copy()

In [ ]:
# GUTCHECK
testip = '99.203.75.4' #10
testip='99.203.75.183' #6
display(df_web_la[df_web_la.ip==testip].drop_duplicates(['ip','rexUrl'])[['ip','rexUrl']])
display(train_la[train_la.ip==testip])
display(test_la[test_la.ip==testip])

,ip,rexUrl
5072094,99.203.75.183,22718-figueroa-st-unit-21
5072097,99.203.75.183,1509-inverness-dr
5072104,99.203.75.183,6954-rubio-ave
5072212,99.203.75.183,16321-janine-dr
5072276,99.203.75.183,9323-alcott-st-apt-101
5072281,99.203.75.183,1415-n-myers-st


,ip,rexUrl
90558,99.203.75.183,22718-figueroa-st-unit-21
90558,99.203.75.183,1509-inverness-dr
90558,99.203.75.183,6954-rubio-ave
90558,99.203.75.183,16321-janine-dr
90558,99.203.75.183,9323-alcott-st-apt-101


,ip,rexUrl
90558,99.203.75.183,1415-n-myers-st


In [ ]:
print(train_la.shape)
print(test_la.shape)
print(test_la.shape[0]/(test_la.shape[0]+train_la.shape[0]))

# There is no IP in test that isnt in train
print(set(train_la.ip) - set(test_la.ip))

# There is no IP in train that isnt in test
print(set(test_la.ip) - set(train_la.ip))

print(train_la.ip.nunique())
print(test_la.ip.nunique())

(51060, 2)
(17102, 2)
0.25090226225756285
set()
set()
8663
8663


In [ ]:
test_g['99.203.75.183']

['1415-n-myers-st']

In [ ]:
train_test_count = {}
for i in test_g.keys():
  trainsize = len(train_g[i])
  testsize = len(test_g[i])
  train_test_count[i] = [trainsize, testsize]

In [ ]:
train_test_count_df = pd.DataFrame.from_dict(train_test_count,orient='index')
train_test_count_df.rename({0:'trainsize',1:'testsize'},axis=1,inplace=True)

In [ ]:
train_test_count_df.head()

,trainsize,testsize
1.0.84.114,1,0
1.125.109.121,1,0
1.125.110.139,1,0
1.128.106.121,2,0
1.128.106.169,1,0


In [ ]:
train_test_count_df.testsize.value_counts(normalize=True).head()

0    0.905910
1    0.059682
2    0.016987
3    0.007755
4    0.002683
Name: testsize, dtype: float64

In [ ]:
# traincount = train_la.groupby('ip').count().reset_index().rename({'rexUrl':'n_listings_train'},axis=1)
# testcount = test_la.groupby('ip').count().reset_index().rename({'rexUrl':'n_listings_test'},axis=1)

# joint = traincount.merge(testcount,on='ip')
# joint = joint.sort_values(['n_listings_train','n_listings_test'],ascending=False).reset_index(drop=True)

In [ ]:
# # temporal group flag
# 
# g = ip_listing.groupby('ip')

# flags = (g.cumcount() + 1) <= np.ceil(g['rexUrl'].transform('size') * 0.6)
# # flags2 = (g.cumcount() + 1) > np.ceil(g['rexUrl'].transform('size') * 0.6)


# train_la = ip_listing.loc[flags]
# blah = ip_listing.loc[np.invert(flags)]

# test_temp = pd.merge(ip_url.drop('score',axis=1), train_la, how='outer', suffixes=('','_y'), indicator=True)
# test_la = test_temp[test_temp['_merge']=='left_only'][ip_url.columns[:-1]]

# train_la = train_la[['ip','rexUrl']].drop_duplicates()
# test_la = test_la[['ip','rexUrl']].drop_duplicates()
# blah = blah[['ip','rexUrl']].drop_duplicates()

## Saving Temporal

In [ ]:
train_la.to_pickle('ip_listing_temporal_train_full.pkl')
test_la.to_pickle('ip_listing_temporal_test_full.pkl')